In [1]:
import scipy.io as sio
import matplotlib.pyplot as plt
import numpy as np
import scipy.linalg as la
## Construct LSTM using Tensorflow + Keras
# Import Libraries
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import SimpleRNN
from tensorflow.keras.layers import Dropout
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from tensorflow.keras import optimizers


######################################################################
######################################################################
######################################################################
## CSP File from: https://github.com/spolsley/common-spatial-patterns

# CSP takes any number of arguments, but each argument must be a collection of trials associated with a task
# That is, for N tasks, N arrays are passed to CSP each with dimensionality (# of trials of task N) x (feature vector)
# Trials may be of any dimension, provided that each trial for each task has the same dimensionality,
# otherwise there can be no spatial filtering since the trials cannot be compared
def CSP(*tasks):
	if len(tasks) < 2:
		print("Must have at least 2 tasks for filtering.")
		return (None,) * len(tasks)
	else:
		filters = ()
		# CSP algorithm
		# For each task x, find the mean variances Rx and not_Rx, which will be used to compute spatial filter SFx
		iterator = range(0,len(tasks))
		for x in iterator:
			# Find Rx
			Rx = covarianceMatrix(tasks[x][0])
			for t in range(1,len(tasks[x])):
				Rx += covarianceMatrix(tasks[x][t])
			Rx = Rx / len(tasks[x])

			# Find not_Rx
			count = 0
			not_Rx = Rx * 0
			for not_x in [element for element in iterator if element != x]:
				for t in range(0,len(tasks[not_x])):
					not_Rx += covarianceMatrix(tasks[not_x][t])
					count += 1
			not_Rx = not_Rx / count

			# Find the spatial filter SFx
			SFx = spatialFilter(Rx,not_Rx)
			filters += (SFx,)

			# Special case: only two tasks, no need to compute any more mean variances
			if len(tasks) == 2:
				filters += (spatialFilter(not_Rx,Rx),)
				break
		return filters

# covarianceMatrix takes a matrix A and returns the covariance matrix, scaled by the variance
def covarianceMatrix(A):
	Ca = np.dot(A,np.transpose(A))/np.trace(np.dot(A,np.transpose(A)))
	return Ca

# spatialFilter returns the spatial filter SFa for mean covariance matrices Ra and Rb
def spatialFilter(Ra,Rb):
	R = Ra + Rb
	E,U = la.eig(R)

	# CSP requires the eigenvalues E and eigenvector U be sorted in descending order
	ord = np.argsort(E)
	ord = ord[::-1] # argsort gives ascending order, flip to get descending
	E = E[ord]
	U = U[:,ord]

	# Find the whitening transformation matrix
	P = np.dot(np.sqrt(la.inv(np.diag(E))),np.transpose(U))

	# The mean covariance matrices may now be transformed
	Sa = np.dot(P,np.dot(Ra,np.transpose(P)))
	Sb = np.dot(P,np.dot(Rb,np.transpose(P)))

	# Find and sort the generalized eigenvalues and eigenvector
	E1,U1 = la.eig(Sa,Sb)
	ord1 = np.argsort(E1)
	ord1 = ord1[::-1]
	E1 = E1[ord1]
	U1 = U1[:,ord1]

	# The projection matrix (the spatial filter) may now be obtained
	SFa = np.dot(np.transpose(U1),P)
	return SFa.astype(np.float32)
######################################################################
def GetMinSteps(indeces, data):
	minVal = 9999;
	for index in indeces:
		length = data[index].shape[1];
		if(length < minVal):
			minVal = length;
	return minVal
######################################################################
def GetData(indeces, dataIn, truncateValue):
	dataOut = []
	truncate = True;
	if truncateValue == 0:
		truncate = False;
	for idx in indeces:
		currentData = dataIn[idx]
		if truncate:
			dataOut.append(currentData[:,0:truncateValue])
		else:
			dataOut.append(currentData)
	return np.asarray(dataOut)
######################################################################
######################################################################
######################################################################

fileNames = ['HaLTSubjectA1602236StLRHandLegTongue.mat',\
'HaLTSubjectA1603086StLRHandLegTongue.mat',\
'HaLTSubjectA1603106StLRHandLegTongue.mat',\
'HaLTSubjectB1602186StLRHandLegTongue.mat',\
'HaLTSubjectB1602256StLRHandLegTongue.mat',\
'HaLTSubjectB1602296StLRHandLegTongue.mat',\
'HaLTSubjectC1602246StLRHandLegTongue.mat',\
'HaLTSubjectC1603026StLRHandLegTongue.mat',\
'HaLTSubjectE1602196StLRHandLegTongue.mat',\
'HaLTSubjectE1602266StLRHandLegTongue.mat',\
'HaLTSubjectE1603046StLRHandLegTongue.mat',\
'HaLTSubjectF1602026StLRHandLegTongue.mat',\
'HaLTSubjectF1602036StLRHandLegTongue.mat',\
'HaLTSubjectF1602046StLRHandLegTongue.mat',\
'HaLTSubjectG1603016StLRHandLegTongue.mat',\
'HaLTSubjectG1603226StLRHandLegTongue.mat',\
'HaLTSubjectG1604126StLRHandLegTongue.mat',\
'HaLTSubjectH1607206StLRHandLegTongue.mat',\
'HaLTSubjectH1607226StLRHandLegTongue.mat',\
'HaLTSubjectI1606096StLRHandLegTongue.mat',\
'HaLTSubjectI1606286StLRHandLegTongue.mat',\
'HaLTSubjectJ1611216StLRHandLegTongue.mat',\
'HaLTSubjectK1610276StLRHandLegTongue.mat',\
'HaLTSubjectK1611086StLRHandLegTongue.mat',\
'HaLTSubjectL1611166StLRHandLegTongue.mat',\
'HaLTSubjectL1612056StLRHandLegTongue.mat',\
'HaLTSubjectM1611086StLRHandLegTongue.mat',\
'HaLTSubjectM1611176StLRHandLegTongue.mat',\
'HaLTSubjectM1611246StLRHandLegTongue.mat']

numDatasets = len(fileNames);

testAccuracyNoCSP = np.zeros((1,numDatasets))
trainAccuracyNoCSP = np.zeros((1,numDatasets))
testAccuracyWithCSP = np.zeros((1,numDatasets))
trainAccuracyWithCSP = np.zeros((1,numDatasets))

for dataset in range(0, numDatasets):

	fileName = fileNames[dataset];

	file = sio.loadmat('../../../matDown/HaLT_Data/{}'.format(fileName)) #replace with .mat file name
	header=file['__header__']
	version=file['__version__']
	glob=file['__globals__']
	#ans=file['ans']


	#x=file['x']
	o=file['o'][0][0]
	data=o['data']
	data = np.transpose(data)
	data = data[0:21,:];
	print(data)
	nS=o['nS'][0][0]
	#values of structure seem to be 2D numpy arrays, if originally a scalar in Matlab.
	#use [0][0] to get scalar.
	#print("Number of samples: {numSamples}".format(numSamples=nS))
	test=o['id'][0] #id value became a 1D array of size 1 for some reason. use [0] to get value
	#print("Dataset ID: {id}".format(id=test))
	chnames=o['chnames'][:,0] #[:,0] converts from 2D array back to 1D array
	#print("Channel names: {channelNames}".format(channelNames=chnames))
	markers = o['marker']
	## The markers are all still individual arrays of size 1x1, so we convert them to an array with single values
	markersArray = []
	for marker in markers:
		markersArray.append(marker[0])
	markersArray = np.asarray(markersArray)

	#################################
	#################################
	#5F interaction paradigm
	#1-thumb MI, 2-index finger MI, 3-middle finger MI, 4-ring finger MI, 5-pinkie finger MI

	#all paradigms
	#99-initial relaxation period
	#91-inter-session rest break period
	#92-experiment end
	#################################

	## Find the starting indeces where the marker changes
	changeIdxs = np.where(np.transpose(markersArray)[:-1] != np.transpose(markersArray)[1:])[0]
	#print("Number of index changes: {idxChanges}".format(idxChanges=changeIdxs.shape[0]))
	## Split the data so that it has its matching marker
	dataSplit = np.array_split(data, changeIdxs[:-1], axis=1)
	splitCount = 0
	for splitData in dataSplit:
		splitCount += 1
	#print("Number of arrays in data split: {num}".format(num=splitCount))
	## Retrieve the marker values for each of the change indeces (changeIdxs)
	markerTargets = markersArray[changeIdxs];
	#print("Number of marker targets: {numTargets}".format(numTargets=markerTargets.shape[0]))

	## To Apply CSP, we first only get the indeces for MI tasks 1 and 2 (left and right hand, respectively.)
	lhIdx = np.where(markerTargets == 1)[0]
	rhIdx = np.where(markerTargets == 2)[0]
	llIdx = np.where(markerTargets == 4)[0]
	tIdx = np.where(markerTargets == 5)[0]
	rlIdx = np.where(markerTargets == 6)[0]
	
	
	
	
	
	#tCount = tIdx.shape
	#print("Thumb Marker Count: {}\tSize of First: ({},{})".format(tCount, dataSplit[tIdx[0]].shape[0],dataSplit[tIdx[0]].shape[1]))
	#print("Thumb Marker Count: {}\tSize of Second: ({},{})".format(tCount, dataSplit[tIdx[1]].shape[0],dataSplit[tIdx[1]].shape[1]))
	#print("Thumb Marker Count: {}\tSize of Third: ({},{})".format(tCount, dataSplit[tIdx[2]].shape[0],dataSplit[tIdx[2]].shape[1]))


	lhIdxMin = GetMinSteps(lhIdx, dataSplit)
	rhIdxMin = GetMinSteps(rhIdx, dataSplit)
	llIdxMin = GetMinSteps(llIdx, dataSplit)
	tIdxMin = GetMinSteps(tIdx, dataSplit)
	rlIdxMin = GetMinSteps(rlIdx, dataSplit)
	minValues = [lhIdxMin, rhIdxMin, llIdxMin, tIdxMin, rlIdxMin]
	#minValues = [lhIdxMin, rhIdxMin]
	#minValues

	#Truncate the data to the min size
	minValue = np.min(minValues)
	print(minValue)

	lhData = GetData(lhIdx, dataSplit, minValue)
	rhData = GetData(rhIdx, dataSplit, minValue)
	llData = GetData(llIdx, dataSplit, minValue)
	tData = GetData(tIdx, dataSplit, minValue)
	rlData = GetData(rlIdx, dataSplit, minValue)
	#print("Length of tData: {}".format(len(tData)))
	#print("Length of iData: {}".format(len(iData)))
	#print("Length of mData: {}".format(len(mData)))
	#print("Length of rData: {}".format(len(rData)))
	#print("Length of pData: {}".format(len(pData)))

	minLen = np.min([len(lhData), len(rhData), len(llData), len(tData), len(rlData)])
	#minLen = np.min([len(lhData), len(rhData)])
	##Want to make sure they are balanced, so we keep the minLen values
	lhData = lhData[0:minLen]
	rhData = rhData[0:minLen]
	llData = llData[0:minLen]
	tData = tData[0:minLen]
	rlData = rlData[0:minLen]


	# In[68]:


	#Construct the target arrays and merge the data
	lhTargets = np.tile(np.array([1,0,0,0,0]),(minLen,1))
	rhTargets = np.tile(np.array([0,1,0,0,0]),(minLen,1))
	llTargets = np.tile(np.array([0,0,1,0,0]),(minLen,1))
	tTargets = np.tile(np.array([0,0,0,1,0]),(minLen,1))
	rlTargets = np.tile(np.array([0,0,0,0,1]),(minLen,1))
    
	#Print the sizes of each data
	#print("lhData shape: {}".format(lhData.shape))

	markerTargets = np.vstack((lhTargets, rhTargets, llTargets, tTargets, rlTargets))
	#markerTargets = np.vstack((lhTargets, rhTargets))
	Data = np.vstack((lhData, rhData, llData, tData, rlData))
	#Data = np.vstack((lhData, rhData))
    
	#Sanity Check
	#print("Data Shape: {arg1}\tmarkerTargets Shape: {arg2}".format(arg1=Data.shape, arg2=markerTargets.shape))

	## Shuffle the data
	Data, markerTargets = shuffle(Data, markerTargets, random_state=0)

	## Split into train and test sets
	DataTrain, DataTest, markerTargetsTrain, markerTargetsTest = train_test_split(Data, markerTargets, test_size=0.3, random_state=1)
	markerTargetsTrain.shape

	## Reshape the data for time-series processing
	## Syntax np.reshape((numExamples, numTimeSteps, numInputs/numFeatures))
	DataTrainRe = DataTrain.reshape((DataTrain.shape[0], DataTrain.shape[1], DataTrain.shape[2]))
	DataTestRe = DataTest.reshape((DataTest.shape[0], DataTest.shape[1], DataTest.shape[2]))

	## Construct the model
	LSTM_EEG = Sequential()
	LSTM_EEG.add(LSTM((100),batch_input_shape=(None,DataTrainRe.shape[1], DataTrainRe.shape[2]), return_sequences=True))
	LSTM_EEG.add(LSTM((50), return_sequences=False))
	LSTM_EEG.add(Dense((5),activation='sigmoid'))

	LSTM_EEG.summary()
	sgd = optimizers.SGD(lr=0.05, decay=1e-6, momentum=0.9, nesterov=True)
	LSTM_EEG.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])


	# In[79]:


	history = LSTM_EEG.fit(DataTrain, markerTargetsTrain, epochs=30,verbose=2, batch_size=16)


	# In[80]:


	predictionsTest = LSTM_EEG.predict(DataTest)


	# In[81]:


	predictionsTest[predictionsTest>0.5] = 1


	# In[82]:


	predictionsTest[predictionsTest <= 0.5] = 0


	# In[83]:


	comparisonArrayTest = predictionsTest == markerTargetsTest


	# In[85]:


	correctCountTest = 0
	for boolValues in comparisonArrayTest:
		if(boolValues[0] & boolValues[1]):
			correctCountTest += 1
	falseCountTest = DataTest.shape[0] - correctCountTest

	predictionsTrain = LSTM_EEG.predict(DataTrain)
	predictionsTrain[predictionsTrain>0.5] = 1;
	predictionsTrain[predictionsTrain<=0.5] = 0;
	comparisonArrayTrain = predictionsTrain == markerTargetsTrain;

	correctCountTrain = 0
	for boolValues in comparisonArrayTrain:
		if(boolValues[0] & boolValues[1]):
			correctCountTrain += 1
	falseCountTrain = DataTrain.shape[0] - correctCountTrain


	# In[87]:
	trainAcc_noCSP = (correctCountTrain*100/DataTrain.shape[0]);
	testAcc_noCSP = (correctCountTest*100/DataTest.shape[0]);

	#Save these values into the array
	trainAccuracyNoCSP[0,dataset] = trainAcc_noCSP;
	testAccuracyNoCSP[0, dataset] = testAcc_noCSP;

	print("#################################")
	print("#################################")
	print("Training Performance:\nCorrect MI Prediction: {}\nIncorrect MI Prediction: {}\nPercent Accuracy: {:.3f}%".format(correctCountTrain, falseCountTrain, trainAcc_noCSP))
	print("#################################")
	print("#################################")
	print("Testing Performance:\nCorrect MI Prediction: {}\nIncorrect MI Prediction: {}\nPercent Accuracy: {:.3f}%".format(correctCountTest, falseCountTest, testAcc_noCSP))
	print("#################################")
	print("#################################")


	# In[88]:


	## Applying CSP to 5F data
	filters = CSP(lhData, rhData, llData, tData, rlData)
	#filters = CSP(lhData, rhData)

	# In[90]:


	filtersArray = np.asarray(filters)


	# In[91]:


	filtersArray.shape


	# In[92]:


	lhData_CSP = np.matmul(np.transpose(filtersArray[0]), lhData)
	rhData_CSP = np.matmul(np.transpose(filtersArray[1]), rhData)
	llData_CSP = np.matmul(np.transpose(filtersArray[2]), llData)
	tData_CSP = np.matmul(np.transpose(filtersArray[3]), tData)
	rlData_CSP = np.matmul(np.transpose(filtersArray[4]), rlData)


	# In[93]:


	Data_CSP = np.vstack((lhData_CSP, rhData_CSP, llData_CSP, tData_CSP, rlData_CSP))
	#Data_CSP = np.vstack((lhData_CSP, rhData_CSP))


	# In[94]:


	#Construct the target arrays and merge the data
	lhTargets = np.tile(np.array([1,0,0,0,0]),(minLen,1))
	rhTargets = np.tile(np.array([0,1,0,0,0]),(minLen,1))
	llTargets = np.tile(np.array([0,0,1,0,0]),(minLen,1))
	tTargets = np.tile(np.array([0,0,0,1,0]),(minLen,1))
	rlTargets = np.tile(np.array([0,0,0,0,1]),(minLen,1))

	markerTargets = np.vstack((lhTargets, rhTargets, llTargets, tTargets, rlTargets))
	#markerTargets = np.vstack((lhTargets, rhTargets))

	# In[95]:


	## Shuffle the data
	Data_CSP, markerTargets_CSP = shuffle(Data_CSP, markerTargets, random_state=0)
	## Split into train and test sets
	DataTrain_CSP, DataTest_CSP, markerTargetsTrain_CSP, markerTargetsTest_CSP = train_test_split(Data_CSP, markerTargets_CSP, test_size=0.3, random_state=1)

	##Use the same model to train and test
	history_CSP = LSTM_EEG.fit(DataTrain_CSP, markerTargetsTrain_CSP, epochs=30,verbose=2, batch_size=16)


	# In[96]:


	predictionsTest = LSTM_EEG.predict(DataTest_CSP)


	# In[97]:


	predictionsTest[predictionsTest>0.5] = 1


	# In[98]:


	predictionsTest[predictionsTest <= 0.5] = 0


	# In[99]:


	comparisonArrayTest = predictionsTest == markerTargetsTest


	# In[100]:


	correctCountTest = 0
	for boolValues in comparisonArrayTest:
		if(boolValues[0] & boolValues[1]):
			correctCountTest += 1
	falseCountTest = DataTest_CSP.shape[0] - correctCountTest

	predictionsTrain = LSTM_EEG.predict(DataTrain_CSP)
	predictionsTrain[predictionsTrain>0.5] = 1;
	predictionsTrain[predictionsTrain<=0.5] = 0;
	comparisonArrayTrain = predictionsTrain == markerTargetsTrain;

	correctCountTrain = 0
	for boolValues in comparisonArrayTrain:
		if(boolValues[0] & boolValues[1]):
			correctCountTrain += 1
	falseCountTrain = DataTrain_CSP.shape[0] - correctCountTrain

	#Computing the accuracy
	trainAcc_wCSP = (correctCountTrain*100/DataTrain_CSP.shape[0]);
	testAcc_wCSP = (correctCountTest*100/DataTest_CSP.shape[0]);

	#Save these values into the array
	trainAccuracyWithCSP[0,dataset] = trainAcc_wCSP;
	testAccuracyWithCSP[0, dataset] = testAcc_wCSP;

	print("#################################")
	print("### RESULTS AFTER APPLYING CSP ##")
	print("#################################")
	print("Training Performance:\nCorrect MI Prediction: {}\nIncorrect MI Prediction: {}\nPercent Accuracy: {:.3f}%".format(correctCountTrain, falseCountTrain, trainAcc_wCSP))
	print("#################################")
	print("#################################")
	print("Testing Performance:\nCorrect MI Prediction: {}\nIncorrect MI Prediction: {}\nPercent Accuracy: {:.3f}%".format(correctCountTest, falseCountTest, testAcc_wCSP))
	print("#################################")
	print("#################################")




[[ -0.    -0.    -0.   ... -36.38 -28.58 -40.94]
 [ -0.    -0.    -0.   ...  -4.69  -5.15  -9.33]
 [ -0.    -0.    -0.   ... -19.24 -20.03 -23.22]
 ...
 [ -0.    -0.    -0.   ...  -7.16  -4.91  -7.68]
 [ -0.    -0.    -0.   ...  -3.36  -2.78  -1.2 ]
 [ -0.    -0.    -0.   ...   3.79   4.74   8.04]]
202
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 21, 100)           121200    
_________________________________________________________________
lstm_1 (LSTM)                (None, 50)                30200     
_________________________________________________________________
dense (Dense)                (None, 5)                 255       
Total params: 151,655
Trainable params: 151,655
Non-trainable params: 0
_________________________________________________________________
Epoch 1/30
34/34 - 1s - loss: 0.5581 - accuracy: 0.2299
Epoch 2/30
34/34 - 0

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:93: ComplexWarning: Casting complex values to real discards the imaginary part


Epoch 1/30
34/34 - 0s - loss: 0.5973 - accuracy: 0.3720
Epoch 2/30
34/34 - 0s - loss: 0.4132 - accuracy: 0.4673
Epoch 3/30
34/34 - 0s - loss: 0.3582 - accuracy: 0.6037
Epoch 4/30
34/34 - 0s - loss: 0.3223 - accuracy: 0.6617
Epoch 5/30
34/34 - 1s - loss: 0.3030 - accuracy: 0.6785
Epoch 6/30
34/34 - 0s - loss: 0.2871 - accuracy: 0.6879
Epoch 7/30
34/34 - 1s - loss: 0.2690 - accuracy: 0.7252
Epoch 8/30
34/34 - 1s - loss: 0.2421 - accuracy: 0.7402
Epoch 9/30
34/34 - 1s - loss: 0.2217 - accuracy: 0.8037
Epoch 10/30
34/34 - 0s - loss: 0.1993 - accuracy: 0.8206
Epoch 11/30
34/34 - 0s - loss: 0.2154 - accuracy: 0.8000
Epoch 12/30
34/34 - 0s - loss: 0.2130 - accuracy: 0.7925
Epoch 13/30
34/34 - 0s - loss: 0.1858 - accuracy: 0.8206
Epoch 14/30
34/34 - 0s - loss: 0.1698 - accuracy: 0.8486
Epoch 15/30
34/34 - 0s - loss: 0.1468 - accuracy: 0.8841
Epoch 16/30
34/34 - 0s - loss: 0.1499 - accuracy: 0.8636
Epoch 17/30
34/34 - 0s - loss: 0.1376 - accuracy: 0.8935
Epoch 18/30
34/34 - 0s - loss: 0.1407 - 

34/34 - 1s - loss: 0.5449 - accuracy: 0.2187
Epoch 2/30
34/34 - 0s - loss: 0.4598 - accuracy: 0.4393
Epoch 3/30
34/34 - 0s - loss: 0.4012 - accuracy: 0.6467
Epoch 4/30
34/34 - 0s - loss: 0.3318 - accuracy: 0.7121
Epoch 5/30
34/34 - 1s - loss: 0.2782 - accuracy: 0.7850
Epoch 6/30
34/34 - 1s - loss: 0.2283 - accuracy: 0.8019
Epoch 7/30
34/34 - 0s - loss: 0.2236 - accuracy: 0.8037
Epoch 8/30
34/34 - 1s - loss: 0.2117 - accuracy: 0.8112
Epoch 9/30
34/34 - 0s - loss: 0.1842 - accuracy: 0.8355
Epoch 10/30
34/34 - 0s - loss: 0.1279 - accuracy: 0.8972
Epoch 11/30
34/34 - 0s - loss: 0.1012 - accuracy: 0.9159
Epoch 12/30
34/34 - 0s - loss: 0.1236 - accuracy: 0.8991
Epoch 13/30
34/34 - 0s - loss: 0.1611 - accuracy: 0.8523
Epoch 14/30
34/34 - 1s - loss: 0.1227 - accuracy: 0.9009
Epoch 15/30
34/34 - 0s - loss: 0.0992 - accuracy: 0.9121
Epoch 16/30
34/34 - 0s - loss: 0.0875 - accuracy: 0.9346
Epoch 17/30
34/34 - 0s - loss: 0.0666 - accuracy: 0.9477
Epoch 18/30
34/34 - 0s - loss: 0.0628 - accuracy: 0

Epoch 15/30
34/34 - 1s - loss: 0.1501 - accuracy: 0.8703
Epoch 16/30
34/34 - 1s - loss: 0.0997 - accuracy: 0.9342
Epoch 17/30
34/34 - 1s - loss: 0.0844 - accuracy: 0.9342
Epoch 18/30
34/34 - 1s - loss: 0.0873 - accuracy: 0.9267
Epoch 19/30
34/34 - 1s - loss: 0.0855 - accuracy: 0.9286
Epoch 20/30
34/34 - 1s - loss: 0.0817 - accuracy: 0.9323
Epoch 21/30
34/34 - 1s - loss: 0.0856 - accuracy: 0.9286
Epoch 22/30
34/34 - 1s - loss: 0.0757 - accuracy: 0.9474
Epoch 23/30
34/34 - 1s - loss: 0.0884 - accuracy: 0.9361
Epoch 24/30
34/34 - 1s - loss: 0.0674 - accuracy: 0.9455
Epoch 25/30
34/34 - 1s - loss: 0.0548 - accuracy: 0.9586
Epoch 26/30
34/34 - 1s - loss: 0.0529 - accuracy: 0.9643
Epoch 27/30
34/34 - 1s - loss: 0.0538 - accuracy: 0.9605
Epoch 28/30
34/34 - 1s - loss: 0.0583 - accuracy: 0.9662
Epoch 29/30
34/34 - 1s - loss: 0.0603 - accuracy: 0.9586
Epoch 30/30
34/34 - 1s - loss: 0.0598 - accuracy: 0.9436
#################################
### RESULTS AFTER APPLYING CSP ##
####################

...

34/34 - 1s - loss: 0.0016 - accuracy: 1.0000
Epoch 28/30
34/34 - 1s - loss: 0.0015 - accuracy: 1.0000
Epoch 29/30
34/34 - 1s - loss: 0.0014 - accuracy: 1.0000
Epoch 30/30
34/34 - 1s - loss: 0.0014 - accuracy: 1.0000
#################################
#################################
Training Performance:
Correct MI Prediction: 535
Incorrect MI Prediction: 0
Percent Accuracy: 100.000%
#################################
#################################
Testing Performance:
Correct MI Prediction: 137
Incorrect MI Prediction: 93
Percent Accuracy: 59.565%
#################################
#################################
Epoch 1/30
34/34 - 1s - loss: 0.5637 - accuracy: 0.3907
Epoch 2/30
34/34 - 1s - loss: 0.3846 - accuracy: 0.5738
Epoch 3/30
34/34 - 1s - loss: 0.3219 - accuracy: 0.6710
Epoch 4/30
34/34 - 1s - loss: 0.2849 - accuracy: 0.7140
Epoch 5/30
34/34 - 1s - loss: 0.2626 - accuracy: 0.7458
Epoch 6/30
34/34 - 1s - loss: 0.2664 - accuracy: 0.7402
Epoch 7/30
34/34 - 1s - loss: 0.2086 - 

202
Model: "sequential_24"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_48 (LSTM)               (None, 21, 100)           121200    
_________________________________________________________________
lstm_49 (LSTM)               (None, 50)                30200     
_________________________________________________________________
dense_24 (Dense)             (None, 5)                 255       
Total params: 151,655
Trainable params: 151,655
Non-trainable params: 0
_________________________________________________________________
Epoch 1/30
34/34 - 1s - loss: 0.5289 - accuracy: 0.2916
Epoch 2/30
34/34 - 1s - loss: 0.4416 - accuracy: 0.5439
Epoch 3/30
34/34 - 1s - loss: 0.3739 - accuracy: 0.6729
Epoch 4/30
34/34 - 1s - loss: 0.3071 - accuracy: 0.7533
Epoch 5/30
34/34 - 1s - loss: 0.2483 - accuracy: 0.8131
Epoch 6/30
34/34 - 1s - loss: 0.2148 - accuracy: 0.8187
Epoch 7/30
34/34 - 1s - loss: 0.2058 

34/34 - 1s - loss: 0.5048 - accuracy: 0.4944
Epoch 2/30
34/34 - 1s - loss: 0.3264 - accuracy: 0.6504
Epoch 3/30
34/34 - 1s - loss: 0.2709 - accuracy: 0.7180
Epoch 4/30
34/34 - 1s - loss: 0.2570 - accuracy: 0.7312
Epoch 5/30
34/34 - 1s - loss: 0.2243 - accuracy: 0.7914
Epoch 6/30
34/34 - 1s - loss: 0.1797 - accuracy: 0.8177
Epoch 7/30
34/34 - 1s - loss: 0.1621 - accuracy: 0.8477
Epoch 8/30
34/34 - 1s - loss: 0.1576 - accuracy: 0.8459
Epoch 9/30
34/34 - 1s - loss: 0.1355 - accuracy: 0.8816
Epoch 10/30
34/34 - 1s - loss: 0.1408 - accuracy: 0.8835
Epoch 11/30
34/34 - 1s - loss: 0.1073 - accuracy: 0.9173
Epoch 12/30
34/34 - 1s - loss: 0.0851 - accuracy: 0.9323
Epoch 13/30
34/34 - 1s - loss: 0.0850 - accuracy: 0.9248
Epoch 14/30
34/34 - 1s - loss: 0.0832 - accuracy: 0.9380
Epoch 15/30
34/34 - 1s - loss: 0.0934 - accuracy: 0.9154
Epoch 16/30
34/34 - 1s - loss: 0.1125 - accuracy: 0.8966
Epoch 17/30
34/34 - 1s - loss: 0.1048 - accuracy: 0.9079
Epoch 18/30
34/34 - 1s - loss: 0.0976 - accuracy: 0

34/34 - 1s - loss: 0.5324 - accuracy: 0.2860
Epoch 2/30
34/34 - 1s - loss: 0.4395 - accuracy: 0.5383
Epoch 3/30
34/34 - 1s - loss: 0.3670 - accuracy: 0.6542
Epoch 4/30
34/34 - 1s - loss: 0.2883 - accuracy: 0.8168
Epoch 5/30
34/34 - 1s - loss: 0.2188 - accuracy: 0.8467
Epoch 6/30
34/34 - 1s - loss: 0.1768 - accuracy: 0.8729
Epoch 7/30
34/34 - 1s - loss: 0.1310 - accuracy: 0.9290
Epoch 8/30
34/34 - 1s - loss: 0.1042 - accuracy: 0.9346
Epoch 9/30
34/34 - 1s - loss: 0.1053 - accuracy: 0.9290
Epoch 10/30
34/34 - 1s - loss: 0.0964 - accuracy: 0.9364
Epoch 11/30
34/34 - 1s - loss: 0.0667 - accuracy: 0.9607
Epoch 12/30
34/34 - 1s - loss: 0.0321 - accuracy: 0.9813
Epoch 13/30
34/34 - 1s - loss: 0.0236 - accuracy: 0.9944
Epoch 14/30
34/34 - 1s - loss: 0.0227 - accuracy: 0.9888
Epoch 15/30
34/34 - 1s - loss: 0.0230 - accuracy: 0.9869
Epoch 16/30
34/34 - 1s - loss: 0.0095 - accuracy: 1.0000
Epoch 17/30
34/34 - 1s - loss: 0.0047 - accuracy: 1.0000
Epoch 18/30
34/34 - 1s - loss: 0.0032 - accuracy: 1

Epoch 15/30
34/34 - 1s - loss: 0.1883 - accuracy: 0.8346
Epoch 16/30
34/34 - 1s - loss: 0.1723 - accuracy: 0.8402
Epoch 17/30
34/34 - 1s - loss: 0.1791 - accuracy: 0.8233
Epoch 18/30
34/34 - 1s - loss: 0.1636 - accuracy: 0.8402
Epoch 19/30
34/34 - 1s - loss: 0.1549 - accuracy: 0.8647
Epoch 20/30
34/34 - 1s - loss: 0.1837 - accuracy: 0.8271
Epoch 21/30
34/34 - 1s - loss: 0.1595 - accuracy: 0.8590
Epoch 22/30
34/34 - 1s - loss: 0.1426 - accuracy: 0.8759
Epoch 23/30
34/34 - 1s - loss: 0.1581 - accuracy: 0.8628
Epoch 24/30
34/34 - 1s - loss: 0.1481 - accuracy: 0.8684
Epoch 25/30
34/34 - 1s - loss: 0.1465 - accuracy: 0.8703
Epoch 26/30
34/34 - 1s - loss: 0.1169 - accuracy: 0.8966
Epoch 27/30
34/34 - 1s - loss: 0.1157 - accuracy: 0.9117
Epoch 28/30
34/34 - 1s - loss: 0.1229 - accuracy: 0.8853
Epoch 29/30
34/34 - 1s - loss: 0.1062 - accuracy: 0.9229
Epoch 30/30
34/34 - 1s - loss: 0.1013 - accuracy: 0.9117
#################################
### RESULTS AFTER APPLYING CSP ##
####################

In [2]:
fileNames

['HaLTSubjectA1602236StLRHandLegTongue.mat',
 'HaLTSubjectA1603086StLRHandLegTongue.mat',
 'HaLTSubjectA1603106StLRHandLegTongue.mat',
 'HaLTSubjectB1602186StLRHandLegTongue.mat',
 'HaLTSubjectB1602256StLRHandLegTongue.mat',
 'HaLTSubjectB1602296StLRHandLegTongue.mat',
 'HaLTSubjectC1602246StLRHandLegTongue.mat',
 'HaLTSubjectC1603026StLRHandLegTongue.mat',
 'HaLTSubjectE1602196StLRHandLegTongue.mat',
 'HaLTSubjectE1602266StLRHandLegTongue.mat',
 'HaLTSubjectE1603046StLRHandLegTongue.mat',
 'HaLTSubjectF1602026StLRHandLegTongue.mat',
 'HaLTSubjectF1602036StLRHandLegTongue.mat',
 'HaLTSubjectF1602046StLRHandLegTongue.mat',
 'HaLTSubjectG1603016StLRHandLegTongue.mat',
 'HaLTSubjectG1603226StLRHandLegTongue.mat',
 'HaLTSubjectG1604126StLRHandLegTongue.mat',
 'HaLTSubjectH1607206StLRHandLegTongue.mat',
 'HaLTSubjectH1607226StLRHandLegTongue.mat',
 'HaLTSubjectI1606096StLRHandLegTongue.mat',
 'HaLTSubjectI1606286StLRHandLegTongue.mat',
 'HaLTSubjectJ1611216StLRHandLegTongue.mat',
 'HaLTSubj

In [3]:
testAccuracyNoCSP

array([[55.65217391, 57.97101449, 62.17391304, 53.94736842, 55.02392344,
        52.17391304, 63.91304348, 57.82608696, 52.60869565, 52.23214286,
        54.38596491, 64.7826087 , 60.43478261, 52.60869565, 63.04347826,
        57.45614035, 65.2173913 , 40.52863436, 51.67464115, 50.69767442,
        50.43859649, 72.60869565, 59.56521739, 56.14035088, 62.60869565,
        64.9122807 , 66.08695652, 62.17391304, 53.07017544]])

In [4]:
testAccuracyWithCSP

array([[73.91304348, 77.29468599, 78.69565217, 75.43859649, 65.07177033,
        62.17391304, 76.52173913, 68.26086957, 74.34782609, 69.64285714,
        68.85964912, 74.34782609, 77.82608696, 73.91304348, 94.7826087 ,
        89.03508772, 86.08695652, 48.89867841, 67.46411483, 73.02325581,
        58.77192982, 91.73913043, 84.7826087 , 65.35087719, 87.39130435,
        87.28070175, 90.        , 81.30434783, 64.03508772]])

In [5]:
trainAccuracyNoCSP

array([[100.        , 100.        ,  99.81308411, 100.        ,
         98.55967078,  99.81308411, 100.        ,  91.96261682,
         99.81308411,  99.23224568,  98.68421053, 100.        ,
        100.        , 100.        ,  96.63551402,  96.2406015 ,
        100.        ,  91.28787879,  87.65432099, 100.        ,
         80.07518797, 100.        , 100.        ,  98.4962406 ,
        100.        , 100.        ,  98.69158879, 100.        ,
         99.62406015]])

In [6]:
trainAccuracyWithCSP

array([[ 98.87850467, 100.        ,  91.58878505,  97.7443609 ,
         98.55967078,  87.6635514 ,  98.5046729 ,  88.03738318,
         97.00934579,  96.92898273,  93.42105263,  99.25233645,
         99.06542056,  97.57009346, 100.        , 100.        ,
         98.69158879,  78.78787879,  89.71193416,  95.6       ,
         81.57894737, 100.        ,  98.5046729 ,  94.73684211,
        100.        ,  99.06015038, 100.        , 100.        ,
         96.61654135]])

In [7]:
combinedArray = np.transpose(np.asarray(np.vstack((trainAccuracyNoCSP,testAccuracyNoCSP,trainAccuracyWithCSP, testAccuracyWithCSP))))

In [8]:
combinedArray.shape

(29, 4)

In [9]:
np.savetxt('HaLTperSubject.csv', combinedArray, delimiter=',')